# Access Estimate

TODO:
- Get proper NTL from website (and mosaic)
- Mosaic all targets
- Loop through countries.gpkg

In [ ]:
from pathlib import Path

from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.warp import reproject, Resampling

from gridfinder._util import save_raster, clip_raster

## Setup

In [ ]:
country = 'Burundi'
access_total = 0.08
access_urban = 0.50
access_rural = 0.02

In [ ]:
# country = 'Uganda'
# access_total = 0.27
# access_urban = 0.58
# access_rural = 0.18

In [ ]:
# country = 'United_Republic_of_Tanzania'
# access_total = 0.33
# access_urban = 0.65
# access_rural = 0.17

In [ ]:
targets_in = f'targets_{country}.tif'
folder = Path.home() / 'Documents/GIS'
aoi_in = folder / 'countries_rough.gpkg'
pop_in = folder / 'GHS_POP_250.tif'
urban_in = folder / 'GHS_URB_RUR.tif'
ntl_in = folder / 'VIIRS_Africa_2016.tif'

## Need to clip all to same AOI

In [ ]:
aoi_big = gpd.read_file(aoi_in)
aoi = aoi_big.loc[aoi_big['ADMIN'] == country.replace('_', ' ')]

In [ ]:
# Clip all to same AOI
pop, affine, crs = clip_raster(pop_in, aoi)
urban, urban_aff, urban_crs = clip_raster(urban_in, aoi)
ntl, ntl_aff, ntl_crs = clip_raster(ntl_in, aoi)
targets, targets_aff, targets_crs = clip_raster(targets_in, aoi)

In [ ]:
pop[pop < 0] = 0
urban[urban < 0] = 0
ntl[ntl < 0] = 0
targets[targets < 0] = 0

## Make all the rasters the same!

In [ ]:
def make_same_as(curr_arr, curr_aff, curr_crs, dest_arr_like, dest_affine, dest_crs):
    dest_arr = np.empty_like(dest_arr_like)

    with rasterio.Env():
        reproject(
            source=curr_arr,
            destination=dest_arr,
            src_transform=curr_aff,
            dst_transform=dest_affine,
            src_crs=curr_crs,
            dst_crs=dest_crs,
            resampling=Resampling.nearest,
        )

    return dest_arr

In [ ]:
urban = make_same_as(urban, urban_aff, urban_crs, pop, affine, crs)
ntl = make_same_as(ntl, ntl_aff, ntl_crs, pop, affine, crs)
targets = make_same_as(targets, targets_aff, targets_crs, pop, affine, crs)
assert pop.shape == urban.shape == ntl.shape == targets.shape

In [ ]:
# Drop 0 pop makes visual inspection easier
# And so that quantile values ignore 0
pop[pop == 0] = np.nan

## Buffer targets

In [ ]:
max_row = targets.shape[0]
max_col = targets.shape[1]
new_targets = targets.copy()

for row in range(0, max_row):
    for col in range(0, max_col):
        loc = (row, col)
        if targets[loc] == 1:
            for i in range(-1, 2):
                for j in range(-1, 2):
                    next_row = row + i
                    next_col = col + j
                    next_loc = (next_row, next_col)
                    
                    if not (
                        next_row < 0
                        or next_col < 0
                        or next_row >= max_row
                        or next_col >= max_col
                        or next_loc == loc
                    ):
                        new_targets[next_loc] = 1
                        
targets = new_targets

## And have a look

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12,12))
axes =     [ax1,   ax2,     ax3,   ax4]
rasters =  [pop,   urban,   ntl,   targets]
titles =   ['pop', 'urban', 'ntl', 'targets']
maxes =    [1000,  None,    1,     None]

for ax, raster, title, vmax in zip(axes, rasters, titles, maxes):
    ax.imshow(raster, vmax=vmax)
    ax.set_title(title)
plt.show()

## Run the algorithm

### Option 1
Iterate, changing `condition_del`, `ntl_cut` and maybe some 'bumping factor' each time?

### Option 2
Use openelec approach...

In [ ]:
more = {
    'urban': {
        0.25: 0.015,
        0.5:  0.02,
        0.75: 0.025,
        1:    0.03
    },
    'rural': {
        0.25: 0.005,
        0.5:  0.01,
        0.75: 0.015,
        1:    0.02
    }
}

runs = 0
while True:
    runs += 1
    # The calculated weights for each segment will go here
    weights = np.zeros_like(pop)

    # Investigate each combination of urban/rural and four quartiles
    # of population density
    for loc in ['urban', 'rural']:
        for q in [0.25, 0.5, 0.75, 1]:

            # Values of 2 and 3 are considered urban
            if loc == 'urban':
                condition_del = urban < 3
                access_level = access_urban
            else:
                condition_del = urban >= 3
                access_level = access_rural

            # Ignore errors from doing arr[arr < x] with nan values
            with np.errstate(invalid='ignore'):
                pop_temp = np.copy(pop)  # local copy of pop for this loop
                pop_temp[condition_del] = np.nan  # remove urban/rural
                pop_temp[targets == 0] = np.nan  # remove not electrified

                # Filter to only keep this quartile
                quant_below = np.nanquantile(pop_temp, q-0.25)
                quant = np.nanquantile(pop_temp, q)
                pop_temp[pop_temp <= quant_below] = np.nan
                pop_temp[pop_temp > quant] = np.nan

                # Get the average brightness per person of the top x% for this quartile
                # Where x is the rural/urban access rate
                ntl_per_pop = ntl / pop_temp
                ntl_cut = np.nanquantile(ntl_per_pop, min(max(1-access_level-more[loc][q], 0), 1))

                # Create a weights array and assign values accoring to the formula below
                w = np.zeros_like(pop)
                w = 1 - (ntl_cut - ntl_per_pop)/ntl_cut
                w[w > 0.95] = 0.95  # limit values to max 1
                w[np.isnan(w)] = 0

                # Add the sucessive weights to the main array
                weights += w

    # Create electrified array by multiplying
    pop_elec = pop * weights
    pop_elec[np.isnan(pop_elec)] = 0
    
    access_model_total = np.nansum(pop_elec) / np.nansum(pop)
    print(access_total, access_model_total)
    
    if abs(access_total - access_model_total) < 0.02:
        print('yay')
        break
    elif runs >= 20:
        print('limit')
        break
    else:
        if access_model_total < access_total:
            direc = 1
        else:
            direc = -1
        for k, v in more.items():
            for l, w in v.items():
                more[k][l] += more[k][l] * direc * (20/runs) * abs(access_total - access_model_total)

## Check and save results

In [ ]:
access_model_total = np.nansum(pop_elec) / np.nansum(pop)
access_model_urban = np.nansum(pop_elec[urban >= 3]) / np.nansum(pop[urban >= 3])
access_model_rural = np.nansum(pop_elec[urban < 3]) / np.nansum(pop[urban < 3])

print('Access\tActual\tModel')
print(f'Total:\t{access_total:.2f}\t{access_model_total:.2f}')
print(f'Urban:\t{access_urban:.2f}\t{access_model_urban:.2f}')
print(f'Rural:\t{access_rural:.2f}\t{access_model_rural:.2f}')

In [ ]:
save_raster(f'pop_elec_{country}.tif', pop_elec, affine, crs)
save_raster(f'weights_{country}.tif', weights, affine, crs)